# 参考カーネル
- https://www.kaggle.com/vjgupta/reach-top-10-with-simple-model-on-housing-prices
- https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

# Stacking Modelとは？
- stacking modelはkaggleで回帰の精度を上げるために作られた手法のよう。
- https://burakhimmetoglu.com/2016/12/01/stacking-models-for-improved-predictions/

In [ ]:
#import some necessary librairies

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

from scipy import stats
from scipy.stats import norm, skew #for some statistics

# データのロード

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

train.describe()

In [ ]:
# IDはモデルの予測に不要のため、一旦落とす
#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

In [ ]:
print ("Size of train data after dropping Id: {}" .format(train.shape))
print ("Size of test data after dropping Id: {}" .format(test.shape))

# 外れ値の対応
- GrLivAreanの外れ値を取ることを、データ作成者が推薦しているため4000以上のものを除外する。
- データ作成者は以下のように言及している。
- I would recommend removing any houses with more than 4000 square feet from the data set (which eliminates these five unusual observations) before assigning it to students.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['GrLivArea'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

In [ ]:
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

# 相関関係の確認
- 相関関係が0.5以上のものを確認する

In [ ]:
# Find correlations with the target and sort
correlations = train.corr()['SalePrice'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(30))
print('\nMost Negative Correlations:\n', correlations.head(30))

In [ ]:
corrmat = train.corr()
top_corr_features = corrmat.index[abs(corrmat["SalePrice"])>0.5]
plt.figure(figsize=(10,10))
g = sns.heatmap(train[top_corr_features].corr(),annot=True,cmap="RdYlGn")

- 以下の項目とSalePriceの相関関係が強いことが分かる
- OverallQual, GrLivArea, TotalBsmtSF, GarageCars, GarageArea, 1stFlrSF
- 上記の内GarageCarsとGarageAreaの相関が強い（まあ止められる車の数と車庫の大きさが大きいのは当たり前かな・・・・）
- TotalBSMTSF（地下室の面積）と1stFlrSF（1階の面積）の相関も高い

In [ ]:
sns.barplot(train.OverallQual,train.SalePrice)

- OverallQual（家の状態）によって価格が明らかに上昇しているのが分かる

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train[cols], size = 2.5)
plt.show();

- 散布図を見るとTotalBsmtSF（地下面積）とGrLivArea（Above ground living area square feet）の形がちょっと変なことが分かる

In [ ]:
sns.scatterplot(train.GrLivArea,train.TotalBsmtSF)

- TotalBsmtSFとGrLivAreaは同じになるケースが多く、TotalBsmtSFの値がGrLivAreaの値を来れることがまれである

## もう少し詳細にSalePriceとFeatureの関係を調べる

# 目的変数の変形

In [ ]:
sns.distplot(train['SalePrice'])
plt.show()

目的変数（SalePrice）をヒストグラムに起こすと、正規分布になっていないことが分かる。そのため対数化する（値の振れ幅を小さくすることでばらつきを小さくする）

In [ ]:
# 対数計算を実施
# 数字のばらつき、偏りを小さくする
train['SalePrice'] = np.log1p(train['SalePrice'])

sns.distplot(train['SalePrice'])
plt.show()

# 特徴量エンジニアリング

In [ ]:
# 全データを一旦結合
# テストも含めた全データで特徴量を見ていく（例：欠損値を補完する際のmedian値など）
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

## 欠損値

### 欠損値の確認

In [ ]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
# TOP30だけを一旦見る
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})

In [ ]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
# 最も欠損値の多いPoolQC（プールの品質）を見る
all_data.PoolQC.loc[all_data.PoolQC.notnull()]

### 欠損値の補完

In [ ]:
# PoolQCにおいてNullの部分はNo Pool（プールがない）という場合。今回はNullの部分をNoneという文字列で埋める
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")

In [ ]:
# MiscFeature(その他の特徴)でのNullはno misc feature
# Alley、Fence、FireplaceQuも同様
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

LotFrontage（間口の広さ）はNeighborhoodに関係ありそうだから、Neighborhoodごとの中間値を欠損値に格納するようにする

In [ ]:
# LotFrontage(間口の広さ)
# Grouping by Neighborhood and Check the LotFrontage. Most of the grouping has similar areas
grouped_df = all_data.groupby('Neighborhood')['LotFrontage']

for key, item in grouped_df:
    print(key,"\n")
    print(grouped_df.get_group(key))
    break


In [ ]:
sns.stripplot(x = train.Neighborhood.values, y = train.LotFrontage.values, jitter=0.1, alpha=0.5)

In [ ]:
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

In [ ]:
# GarageType、GarageFinish、GarageQual、GarageCondはNoneで埋める
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    all_data[col] = all_data[col].fillna('None')

In [ ]:
abc = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','GarageYrBlt', 'GarageArea', 'GarageCars']
all_data.groupby('GarageType')[abc].count()

In [ ]:
#GarageYrBlt, GarageArea and GarageCarsは0で埋める
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)

In [ ]:
#BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath and BsmtHalfBathも0で埋める
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

In [ ]:
#BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2 がnullということは地下室に関係する特徴がないということなので、Noneで埋める
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

In [ ]:
#MasVnrArea、MasVnrTypeは石造りのエリアのサイズなので、NAの場合ないということで、0を埋める
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

In [ ]:
#MSZoningはRLが最も多い値なのでRLで埋める
all_data['MSZoning'].value_counts()

In [ ]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

In [ ]:
#Utilitiesはほどんどの値で同じAllPubであり、あまり意味がないので列ごと削除する
all_data['Utilities'].value_counts()

In [ ]:
all_data = all_data.drop(['Utilities'], axis=1)

In [ ]:
# Functionalについてドキュメントで、どうやらNAの場合はtypicalと書いてあるらしい
all_data["Functional"] = all_data["Functional"].fillna("Typ")

In [ ]:
#Electrical,KitchenQual, Exterior1st, Exterior2nd, SaleTypeもカテゴリー型のため、最頻値で埋めるようにする
mode_col = ['Electrical','KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType']
for col in mode_col:
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

In [ ]:
# MSSubClassはNAは建物のクラスがないということなので、Noneで穴埋め
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

### 欠損値がもうないか確認する

In [ ]:
#Check remaining missing values if any 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

### 数値型のカテゴリー値の対応
数値型のカテゴリーをきちんとカテゴリー型に変換しない場合、モデルによっては数値の大小関係を判断してしまう。
そのためちゃんと変換が必要

In [ ]:
all_data['OverallCond'].value_counts()

In [ ]:
#MSSubClass=The building class
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

面積関係の値は不動産価格に大きく影響するため、全面積の合計の特徴量を新しく作成する

In [ ]:
# Adding total sqfootage feature 
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
# 相関の強い2つを掛け合わせる
# https://www.kaggle.com/katotaka/prediction-house-prices-eda
all_data["Interaction"] = all_data["TotalSF"] * all_data["OverallQual"]

# object型以外の特徴量の偏りを見る

In [ ]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(15)

実データを扱う際、正規分布になっておらず、ゆがんだデータも多く存在する。そのため正規分布に近づけるためBox-Cox変換を利用する

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)

# Dummy-Encoding
カテゴリー型の値をone-hotencodingで変換する

In [ ]:
all_data = pd.get_dummies(all_data)
all_data.shape

# モデルの作成

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]
train.shape

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

## クロスバリデーション用の関数

In [ ]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

### Ridge Regression

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

### Lasso Regression

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
score = rmsle_cv(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

### Elastic Net Regression

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


### Gradient Boosting Regression

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

### XGBoost

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

### LightGBM

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

# Stacking Model

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

## Enet,KRR,Gboost,XGB,LightGBMを平均化し、メタモデルとしてlassoを利用

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR, model_xgb, model_lgb),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))


## StackedRegressor, XGBoost,LightGBMのアセンブル

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

## 最終トレーニングと予測

### StackedRegressor

In [ ]:
stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = stacked_pred
sub.to_csv('submission.csv',index=False)